In [82]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import PillowWriter, FuncAnimation

from src.io.datasets import CSVReader
from src.config.settings import get_settings

%matplotlib widget

In [83]:
settings = get_settings()

reader = CSVReader()

In [84]:
prob_blitz = reader.read(settings.FRAME_DISGUISE_RESULTS_FILE)

In [85]:
prob_blitz = prob_blitz[(prob_blitz.gameId == 2022102000) & (prob_blitz.playId == 4205)].sort_values(by="frameId")
prob_blitz['time'] = prob_blitz['time'].apply(lambda x: pd.to_datetime(x) if '.' in x else pd.to_datetime(x + '.000'))

budda_baker_blitz_prob = prob_blitz[(prob_blitz.jerseyNumber == 3)]
byron_murphy_blitz_prob = prob_blitz[(prob_blitz.jerseyNumber == 7)]

ERROR:tornado.application:Exception in callback <bound method TimerBase._on_timer of <matplotlib.backends.backend_webagg_core.TimerTornado object at 0x7fba050e3d40>>
Traceback (most recent call last):
  File "/home/miguelmd/projects/big-data-bowl-2025/.venv/lib/python3.12/site-packages/tornado/ioloop.py", line 937, in _run
    val = self.callback()
          ^^^^^^^^^^^^^^^
  File "/home/miguelmd/projects/big-data-bowl-2025/.venv/lib/python3.12/site-packages/matplotlib/backend_bases.py", line 1179, in _on_timer
    ret = func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^
  File "/home/miguelmd/projects/big-data-bowl-2025/.venv/lib/python3.12/site-packages/matplotlib/animation.py", line 1407, in _step
    still_going = super()._step(*args)
                  ^^^^^^^^^^^^^^^^^^^^
  File "/home/miguelmd/projects/big-data-bowl-2025/.venv/lib/python3.12/site-packages/matplotlib/animation.py", line 1101, in _step
    self._draw_next_frame(framedata, self._blit)
  File "/home/miguelmd/proje

In [86]:
budda_baker_blitz_prob

,gameId,playId,nflId,jerseyNumber,club,position,displayName,frameId,time,wasInitialPassRusher,blitz_probability,frame_disguise,num_frames
422012,2022102000,4205,44848,3,ARI,14,Budda Baker,6,2022-10-21 03:28:48.600,0,0.925386,0.925386,74
422013,2022102000,4205,44848,3,ARI,14,Budda Baker,7,2022-10-21 03:28:48.700,0,0.948286,0.948286,74
422014,2022102000,4205,44848,3,ARI,14,Budda Baker,8,2022-10-21 03:28:48.800,0,0.878304,0.878304,74
422015,2022102000,4205,44848,3,ARI,14,Budda Baker,9,2022-10-21 03:28:48.900,0,0.907401,0.907401,74
422016,2022102000,4205,44848,3,ARI,14,Budda Baker,10,2022-10-21 03:28:49.000,0,0.962277,0.962277,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...
422081,2022102000,4205,44848,3,ARI,14,Budda Baker,75,2022-10-21 03:28:55.500,0,0.983857,0.983857,74
422082,2022102000,4205,44848,3,ARI,14,Budda Baker,76,2022-10-21 03:28:55.600,0,0.984707,0.984707,74
422083,2022102000,4205,44848,3,ARI,14,Budda Baker,77,2022-10-21 03:28:55.700,0,0.984707,0.984707,74
422084,2022102000,4205,44848,3,ARI,14,Budda Baker,78,2022-10-21 03:28:55.800,0,0.984707,0.984707,74


In [87]:
kyler_gordon_blitz_prob = budda_baker_blitz_prob.copy()
jaquan_brisker_blitz_prob = byron_murphy_blitz_prob.copy()

# Calculate the time in seconds
kyler_gordon_blitz_prob['time_seconds'] = (
    kyler_gordon_blitz_prob['time'] - kyler_gordon_blitz_prob['time'].iloc[0]
).dt.total_seconds()
time = kyler_gordon_blitz_prob['time_seconds'].values
kyler_gordon = kyler_gordon_blitz_prob['blitz_probability'].values
jaquan_brisker = jaquan_brisker_blitz_prob['blitz_probability'].values

# Colors for the lines
kyler_color = "#2a9d8f"
jaquan_color = "#e76f51"

# Create the figure and axes
fig, ax = plt.subplots()

# Initialize the lines
kyler_gordon_line, = ax.plot([], [], label="Budda Baker (#3)", lw=2, color=kyler_color)
jaquan_brisker_line, = ax.plot([], [], label="Byron Murphy (#7)", lw=2, color=jaquan_color)

# Configure the axes
ax.set_xlabel('Time after Lineset (s)', fontsize=12)
ax.set_ylabel('Blitz Probability', fontsize=12)
ax.set_xlim([0, max(time)])
ax.set_ylim([0, 1])
ax.legend(loc="lower left", bbox_to_anchor=(0, 0.1))

# Update function for the animation
def update(frame):
    kyler_gordon_line.set_data(time[:frame], kyler_gordon[:frame])
    jaquan_brisker_line.set_data(time[:frame], jaquan_brisker[:frame])
    return kyler_gordon_line, jaquan_brisker_line

# Create the animation
fig.tight_layout()
frames_with_pause = list(range(len(time))) + [len(time)] * 65 # Add 35 frames
ani = FuncAnimation(fig=fig, func=update, frames=frames_with_pause, interval=100)

# Save the animation as a GIF
ani.save("2022102000_4205_blitz_probs.gif", writer=PillowWriter(fps=10, bitrate=1800))
plt.close(fig)